<a href="https://colab.research.google.com/github/VindhyaSRajan/Master-s-Thesis/blob/master/LDA_AmazonSummarytData_LawnAndGarden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pandas nltk gensim pyLdavis

     |████████████████████████████████| 1.6MB 8.8MB/s 
  Created wheel for pyLdavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=0537e72378ee610161d6c32956c633275d5e358a48555992a6c5cee9d019af14
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLdavis


## All the Imports go here

In [0]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pyLDAvis.gensim # To visualise LDA model effectively
import pandas as pd
from gensim.models import CoherenceModel #Put in use to calculate the Coherency score for LDA

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Reading the Amazon Product Review file 

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')



In [6]:
df = pd.read_csv('/content/drive/My Drive/reviews_samples_amazonSummary.csv')

df.head()
df.shape

(993490, 10)

In [7]:
df.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,A2VNYWOPJ13AFP,0981850006,"Amazon Customer ""carringt0n""","[6, 7]",This was a gift for my other husband. He's ma...,5.0,Delish,1259798400,"12 3, 2009"
1,1,A20DWVV8HML3AW,0981850006,Cyndy,"[0, 0]","My husband rarely asks for anything specific, ...",5.0,Just what my husband wanted,1371081600,"06 13, 2013"
2,2,A3RVP3YBYYOPRH,0981850006,John Warren,"[9, 11]",This guy knows his stuff. I love the seriously...,5.0,Love This Show!,1257984000,"11 12, 2009"
3,3,A28XY55TP3Q90O,0981850006,"JSG ""Lover of Learning""","[1, 1]","THIS CAN BE FOUND AT: [...] FOR $19.99, also a...",5.0,"BUY ELSEWHERE, TOO EXPENSIVE",1314144000,"08 24, 2011"
4,4,A3VZW1BGUQO0V3,0981850006,"Marc G. Belisle ""Allswell2001""","[2, 2]",Quite good particularly for inspiration (DVD g...,5.0,Quite good but I *much* prefer the cookbooks,1308268800,"06 17, 2011"


In [0]:
df.to_csv('reviews_samples_amazonSummary.csv')
!cp reviews_samples_amazonSummary.csv drive/My\ Drive/

In [0]:
from itertools import islice
fileContents = defaultdict(list)
with open('/content/drive/My Drive/reviews_samples_amazonSummary.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in islice(reader, 50000): # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            fileContents[k].append(v) # append the value into the appropriate list

In [9]:
reviews = fileContents['summary']
type(reviews)

list

In [0]:
reviews = [re.sub(r'[^\w\s]','',str(item)) for item in reviews]

In [0]:
stopwords = set(stopwords.words('english'))
#stopwords.extend(['ok','doesnt','try','go','get','make','dont','2','first','second','place',
                  #'would','really','like','great','came','got'])

In [0]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in reviews]

##LDA Implementation

In [0]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [0]:
dictionary = corpora.Dictionary(texts)

In [15]:
print(dictionary)

Dictionary(5335 unique tokens: ['husband', 'wanted', 'love', 'show', 'buy']...)


In [0]:
#df.dtypes
summary= df['summary'].astype(str).copy() #Need the column as a string
summary= summary.apply(lambda x: x.split())

In [0]:
# So what does the dictionary look like : Dictionary(5335 unique tokens: ['husband', 'wanted', 'love', 'show', 'buy']...)

In [0]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [0]:
#DO NOT RUN THIS ONE. It is for Version 1
#NUM_TOPICS = 10 # This is an assumption. Needs to be tuned. 
#ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=20)#This might take some time.

In [0]:
#trying to log the model fitting and saving it
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import warnings
import gensim
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=corpus,
                           num_topics=20,
                           id2word=dictionary,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

In [22]:
coherence_model_lda = CoherenceModel(model=lda_train4, texts=summary, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

0.46935824437412244

In [23]:
topics = lda_train4.show_topics()
for topic in topics:
    print(topic)

(2, '0.051*"small" + 0.048*"years" + 0.045*"get" + 0.036*"still" + 0.033*"set" + 0.032*"design" + 0.030*"one" + 0.023*"yard" + 0.023*"broke" + 0.022*"old"')
(0, '0.106*"garden" + 0.057*"worth" + 0.044*"happy" + 0.028*"cart" + 0.026*"powerful" + 0.026*"pleased" + 0.020*"stool" + 0.020*"shovel" + 0.020*"life" + 0.019*"money"')
(11, '0.171*"nice" + 0.097*"job" + 0.075*"axe" + 0.066*"really" + 0.035*"right" + 0.030*"solid" + 0.027*"working" + 0.021*"gets" + 0.019*"done" + 0.018*"put"')
(6, '0.084*"sprayer" + 0.056*"metal" + 0.043*"much" + 0.036*"need" + 0.034*"gilmour" + 0.026*"chain" + 0.018*"spray" + 0.018*"hammock" + 0.016*"satisfied" + 0.016*"better"')
(13, '0.168*"best" + 0.118*"grill" + 0.065*"ever" + 0.057*"weber" + 0.026*"box" + 0.023*"ive" + 0.022*"hummingbird" + 0.022*"used" + 0.020*"charcoal" + 0.020*"bag"')
(16, '0.122*"pool" + 0.105*"like" + 0.077*"awesome" + 0.039*"fine" + 0.038*"pump" + 0.027*"service" + 0.022*"customer" + 0.022*"hunter" + 0.020*"bounty" + 0.019*"saw"')
(18,

In [25]:
word_dict = {};
for i in range(lda_train4.num_topics):
    words = lda_train4.show_topic(i, topn = 10)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,garden,good,small,product,mower,love,sprayer,easy,excellent,trap,well,nice,buy,best,better,ok,pool,stars,works,great
1,worth,perfect,years,great,lawn,work,metal,use,cover,replacement,works,job,electric,grill,poor,blower,like,bird,great,money
2,happy,light,get,price,hose,worked,much,fiskars,feeder,purchase,time,axe,dont,ever,expected,fits,awesome,feeder,fun,tool
3,cart,far,still,quality,handy,toro,need,disappointed,snow,needs,made,really,gas,weber,black,piece,fine,bad,splitting,grill
4,powerful,fit,set,birds,last,one,gilmour,cant,little,mole,first,right,cheap,box,decker,2,pump,five,grass,little
5,pleased,big,design,smoker,sprinkler,doesnt,chain,would,sturdy,squirrels,super,solid,advertised,ive,needed,flies,service,review,machine,value
6,stool,pretty,one,flimsy,long,gift,spray,storage,exactly,deck,patio,working,water,hummingbird,thing,less,customer,fantastic,new,detector
7,shovel,heater,yard,useless,durable,didnt,hammock,okay,heavy,fly,even,gets,intex,used,junk,mr,hunter,back,power,waste
8,life,thrower,broke,bit,useful,size,satisfied,1800,beware,never,got,done,trimmer,charcoal,amazing,junk,bounty,squirrel,bought,wonderful
9,money,must,old,loved,reel,3,better,chimney,gardening,pruner,5,put,wow,bag,idea,sure,saw,problem,enough,plastic


In [26]:
lda_display = pyLDAvis.gensim.prepare(lda_train4, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
